<a href="https://colab.research.google.com/github/prathammehta16/Customer_Churn_Prediction_Pyspark/blob/main/Customer_Churn_Prediction_Using_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=45bfd61a40c8b9c5dea1ba1db916a31f6ef08e7aaad28e13a90ac57f4e5ddc56
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
import pyspark

sc = pyspark.SparkContext()
sc.parallelize([1, 2, 3, 4]).sum()


10

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CustomerChurnPrediction').getOrCreate()

In [5]:
spark

In [52]:
df = spark.read.csv("/content/CustomerChurn1.csv", header=True, inferSchema=True)
df.show(5)

+---------+-----------+--------------+-------+----------+------+-------------+----------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------------+-----------------+--------------------+---------------+-------------+-----+
|LoyaltyID|Customer ID|Senior Citizen|Partner|Dependents|Tenure|Phone Service|  Multiple Lines|Internet Service|Online Security|Online Backup|Device Protection|Tech Support|Streaming TV|Streaming Movies|      Contract|Paperless Billing|      Payment Method|Monthly Charges|Total Charges|Churn|
+---------+-----------+--------------+-------+----------+------+-------------+----------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------------+-----------------+--------------------+---------------+-------------+-----+
|   318537| 7590-VHVEG|            No|    Yes|        No|     1|           No|No phone service|             DSL|      

In [53]:
df.printSchema()

root
 |-- LoyaltyID: integer (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Senior Citizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Phone Service: string (nullable = true)
 |-- Multiple Lines: string (nullable = true)
 |-- Internet Service: string (nullable = true)
 |-- Online Security: string (nullable = true)
 |-- Online Backup: string (nullable = true)
 |-- Device Protection: string (nullable = true)
 |-- Tech Support: string (nullable = true)
 |-- Streaming TV: string (nullable = true)
 |-- Streaming Movies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- Paperless Billing: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Monthly Charges: double (nullable = true)
 |-- Total Charges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [54]:
from pyspark.sql.functions import count, when, isnan, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+-----------+--------------+-------+----------+------+-------------+--------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------+-----------------+--------------+---------------+-------------+-----+
|LoyaltyID|Customer ID|Senior Citizen|Partner|Dependents|Tenure|Phone Service|Multiple Lines|Internet Service|Online Security|Online Backup|Device Protection|Tech Support|Streaming TV|Streaming Movies|Contract|Paperless Billing|Payment Method|Monthly Charges|Total Charges|Churn|
+---------+-----------+--------------+-------+----------+------+-------------+--------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------+-----------------+--------------+---------------+-------------+-----+
|        0|          0|             0|      0|         0|     0|            0|             0|               0|              0|            0|                0|  

In [55]:
from pyspark.sql.functions import mean
mean_val = df.select(mean(df['Total Charges'])).collect()[0][0]
df = df.na.fill({'Total Charges': mean_val})

In [56]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

categorical_columns = ['Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Total Charges', 'Churn']

In [57]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in categorical_columns]

encoders = [OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol=indexer.getOutputCol()+"_encoded") for indexer in indexers]

In [58]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = indexers+encoders)

df_transformed = pipeline.fit(df).transform(df)

In [59]:
categorical_columns1 = ['Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Total Charges']
for column in categorical_columns1:
  df_transformed=df_transformed.drop(column)

In [60]:
df_transformed.show()

+---------+-----------+------+---------------+-----+--------------------+-------------+----------------+-------------------+--------------------+----------------------+---------------------+-------------------+-----------------------+------------------+------------------+----------------------+--------------+-----------------------+--------------------+-------------------+-----------+----------------------------+---------------------+------------------------+---------------------------+----------------------------+------------------------------+-----------------------------+---------------------------+-------------------------------+--------------------------+--------------------------+------------------------------+----------------------+-------------------------------+----------------------------+---------------------------+-------------------+
|LoyaltyID|Customer ID|Tenure|Monthly Charges|Churn|Senior Citizen_index|Partner_index|Dependents_index|Phone Service_index|Multiple Lines_in

In [61]:
# Rename the indexed and encoded columns
for indexer,encoder in zip(indexers, encoders):
  column = indexer.getInputCol()
  df_transformed=df_transformed.withColumnRenamed(indexer.getOutputCol()+"_encoded", column+"_encoded")

In [62]:
df_transformed.show()

+---------+-----------+------+---------------+-----+--------------------+-------------+----------------+-------------------+--------------------+----------------------+---------------------+-------------------+-----------------------+------------------+------------------+----------------------+--------------+-----------------------+--------------------+-------------------+-----------+----------------------+---------------+------------------+---------------------+----------------------+------------------------+-----------------------+---------------------+-------------------------+--------------------+--------------------+------------------------+----------------+-------------------------+----------------------+---------------------+-------------+
|LoyaltyID|Customer ID|Tenure|Monthly Charges|Churn|Senior Citizen_index|Partner_index|Dependents_index|Phone Service_index|Multiple Lines_index|Internet Service_index|Online Security_index|Online Backup_index|Device Protection_index|Tech Supp

In [63]:
training_data, test_data = df_transformed.randomSplit([0.8,0.2])

In [64]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="Churn_index", featuresCol="features")

In [65]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[col for col in df_transformed.columns if col not in ["Customer ID", "Churn", "Churn_index"]], 
                                               outputCol="features")

In [66]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[assembler,lr])

model = pipeline.fit(training_data)

In [67]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="Churn_index")
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.9999946389177047
